<a href="https://colab.research.google.com/github/gihoonpark/Don-t-overfit-2-Kaggle/blob/main/Don't_overfit!!_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load data & Simple EDA

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/My Drive/dataset/Dont_overfit!_2/train.csv')
df

In [ ]:
X = df.loc[:,"0":]
y = df.loc[:,"target"]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df[df.columns[2:]].std())

In [ ]:
plt.hist(df[df.columns[2:]].mean())

In [ ]:
import seaborn as sns

sns.heatmap(df.corr())

In [ ]:
X.std(axis=1)

PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.90)
X_pca = pca.fit_transform(X)
X_pca.shape

XGB classification

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
kfold = StratifiedKFold(n_splits = 11, random_state=18)
xgb = XGBClassifier(objective='binary:logistic')
score = cross_validate(xgb, X_pca,y, scoring='accuracy', cv=kfold)
sum(score['test_score'])/len(score['test_score'])

In [ ]:
print('cv_mean_score:',sum(score['test_score'])/len(score['test_score']))
print('cv_std', score['test_score'].std())

In [ ]:
plt.boxplot(score['test_score'])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=[]
for train_index, val_index in kfold.split(X_pca, y):
    xgb = XGBClassifier(objective='binary:logistic')
    X_train = X_pca[train_index]
    y_train = y[train_index]
    X_val = X_pca[val_index]
    y_val = y[val_index]
    xgb.fit(X_train, y_train)
    accuracy.append(accuracy_score(xgb.predict(X_val), y_val))

In [ ]:
print(sum(accuracy)/len(accuracy))

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression().fit(X_pca, y)
params ={'max_iter':[10000],
         'solver':['newton-cg','lbfgs', 'liblinear', 'sag', 'saga']}
grid = GridSearchCV(lr, params, scoring='roc_auc', cv=kfold)
grid.fit(X_pca, y)
best_lr = grid.best_estimator_
print(best_lr)
for train_index, val_index in kfold.split(X_pca, y):
    X_train = X_pca[train_index]
    y_train = y[train_index]
    X_val = X_pca[val_index]
    y_val = y[val_index]
    best_lr.fit(X_train, y_train)
    accuracy.append(accuracy_score(best_lr.predict(X_val), y_val))

In [ ]:
print(sum(accuracy)/len(accuracy))